# 有限射影平面神経衰弱
**有限射影平面神経衰弱**とは, 
「それまでに取られたn組のトランプと矛盾しない有限射影平面上の直線をなすトランプの組を取る」というゲームです。
簡単にいうと、
+ 4枚すべての数が異なる
+ それまでの組(4枚1組)のいずれ(の組)とも、1枚だけ数が一致する

ような4枚組を取る神経衰弱です。

**PlayngCards** という **Pkg** でトランプのカードを扱えるのでだけど, どうにも使えないので, 普通に試してみましょう。

配列を **シャッフル** するのに **Shuffle** という **Pkg** が要るようなので, 導入 **add** しておいて, 用意します。

In [1]:
using Shuffle

当初は, スート(♠♣♡♢)の区別はしていませんでした。が折角なので区別してみる仕様にしました。

In [11]:
suit=["♠","♣","♡","♢"]# スートを文字列の配列で, カードは配列で[数,スート]としています。
scard=["A","2","3","4","5","6","7","8","9","X","J","Q","K"]# 数です。1は A(エース), 11,12,13は JQK です。10だけ2桁は面倒なので X としてます。
function card(c)# カードの配列から, "スート数"という文字列を返します。
    suit[c[2]]*scard[c[1]]
end
deck=[]# トランプ１セットのことをデッキと呼ぶようです。
for i=1:4,j=1:13# スート毎に1から13までをデッキに用意します。
    push!(deck,[j,i])
end
deck=shuffle(deck)# シャッフルします。
#println(join(deck,","))
deckc=[]
for c ∈ deck
    push!(deckc,card(c))
end
for i=1:4
    println(join(deckc[1+(i-1)*13:13*i],","))
end

♠4,♡X,♣7,♡2,♠K,♢A,♢4,♡4,♣8,♣X,♠J,♡8,♣A
♢7,♢2,♡3,♡J,♢Q,♣K,♢J,♡5,♣4,♣2,♣Q,♠5,♢8
♠6,♠2,♠Q,♠7,♡Q,♡6,♢X,♣6,♣3,♢K,♣5,♠X,♠A
♠3,♢6,♡7,♠9,♠8,♣9,♢3,♢5,♣J,♡9,♢9,♡K,♡A


In [71]:
function handnum(hand)# 4枚組から, 数だけの配列を返します。
    [hand[1][1],hand[2][1],hand[3][1],hand[4][1]]
end
function cardp(crdset)# [数,スート]というカードの配列の配列 crdset をスートの絵柄と数の文字列にして返します。 
    crdc=[]
    for c ∈ crdset
        push!(crdc,card(c))
    end
    join(crdc,"")
end
function cardpl(crdsetlst)# [数,スート]というカードの配列の配列 crdset の配列 crdsetlst を()で区切ってスートの絵柄と数の文字列にして返します。
    crdcl=[]
    for c ∈ crdsetlst
        push!(crdcl,"("*cardp(c)*")")
    end
    join(crdcl,",")
end
function chkHand(hc,hsc)# 取ろうとしている4枚組が, 2つの条件を満たしているかどうかチェックします。
    # hcはカード[数,スート]4枚の組の配列。hscは取ったカード[数,スート]の組の配列
    h=handnum(hc)# 取ろうとしている4枚の数の配列を h に
    ans=true
    if length(unique(h))!=4# 取ろうとしている 4枚 に同じ数があれば false を返す。
        ans=false
    else
        hs=[]# 既に取った分の4枚組の配列から数の配列にしたものを hs。hsは, 「4つの数の配列」の配列
        for hss ∈ hsc# 1組ずつ数の配列に変えて hs に収める。
            push!(hs,handnum(hss))
        end
        if hs==[]# まだ1組も取っていなければ true
            ans
        else
            for ha ∈ hs# 既に取った4枚組を1つずつチェックする。
                if length(Set(h) ∩ Set(ha))!=1# 集合で評価して, 共通部分が 1 つであれば良し。
                    ans=false# 駄目なら false でループを抜ける。
                    break
                end
            end
        end
    end
    ans# true or false
end
for ggg=1:10# 何度かこのゲームを繰り返す。
    deck=[]# 毎回デッキの準備から。
    for i=1:4,j=1:13
        push!(deck,[j,i])
    end
    hands=[]
    for gg=1:300# 4枚ずつ調べていって, 取れる分は取る。取れなかった分は, ndeck に収めて, それをデッキとして繰り返す。
        deck=shuffle(deck)# シャッフルする。
        ndeck=[]# 
        for g=1:length(deck)/4# デッキを4枚毎に調べる。
            handc=[]# handc は次に取る4枚組
            for i=1:4# 1枚ずつデッキから取り出して, handcに入れていく
                push!(handc,popfirst!(deck))# push!は配列に入れる。popfirst!はデッキの先頭から1枚取り除いたもの
            end# これで 4枚 が準備できる。
#            println(handc,cardp(handc))
#            println(cardp(handc))
#    println(join(deck,","))
#    println(join(hand,","))
#            if length(unique(handnum(handc)))==4 && chkHand(handc,hands)
            if chkHand(handc,hands)# 条件が合えば, 
                push!(hands,handc)# 1組取れたことにする。取れた組の配列 hands に加える。
            else# 駄目なら
#                println(cardpl(ndeck),",",handc)
#                println(cardp(ndeck),", ",cardp(handc))
                ndeck=[ndeck;handc]# 次のデッキ ndeck に handc を合わせて元に戻す
            end
        end
#        println(join(hlst,","))
#    println(join(ndeck,","))
        if length(ndeck)==0
            println(gg)
            break
        else
            deck=ndeck# 取れなかった分のカードで次の回へ
        end
    end
    hlst=[]
    for h ∈ hands
        push!(hlst,"("*join(cardp(h),"")*")")
    end
    if length(hlst)==13
        print("OK > ")
        println(join(hlst,""))
    else
#        print(length(hlst),", ")
    end
end

196
OK > (♣A♠5♣X♢6)(♣J♠6♡7♣3)(♢J♢2♠K♣5)(♢Q♡8♠X♡J)(♢K♣4♣6♢8)(♢4♢5♣Q♢7)(♡Q♣K♡3♠A)(♡A♣2♠7♣8)(♢X♠4♢3♡2)(♢9♣7♡X♡K)(♠9♠8♠3♡5)(♣9♡4♠J♢A)(♡9♠2♡6♠Q)
117
OK > (♠9♡6♢4♠5)(♡J♣8♡5♣X)(♣9♢2♣J♣K)(♡4♣A♡X♠2)(♢5♡2♠3♢Q)(♣Q♢9♡7♢X)(♣5♢7♠A♠K)(♣6♠J♢A♡Q)(♠6♢K♢3♠X)(♣2♣7♢6♡8)(♠7♡3♠4♢J)(♣4♢8♠Q♡K)(♠8♣3♡9♡A)
93
OK > (♣K♡9♢X♣J)(♠2♠K♠4♣8)(♢Q♠8♠9♠3)(♣X♣Q♠5♣4)(♢4♣7♣9♠A)(♢6♡4♡J♡3)(♡6♡X♡7♢8)(♠Q♢A♠6♢K)(♣3♡A♣2♠X)(♡Q♠J♡2♠7)(♣5♢J♣A♡8)(♢3♢7♡5♡K)(♣6♢9♢5♢2)
144
OK > (♡J♢K♡Q♠5)(♢Q♡7♢X♡A)(♣K♢8♠X♡3)(♣A♢2♡K♠4)(♠A♢9♣J♣3)(♣2♢5♣9♣X)(♣7♣4♡5♠3)(♠7♢J♡2♣8)(♠6♠K♢7♠9)(♡X♢4♠J♣6)(♠8♣Q♡9♡4)(♡6♣5♢A♡8)(♠2♠Q♢3♢6)
107
OK > (♠J♣8♣3♣A)(♣4♢5♢8♡K)(♣6♠7♣X♡8)(♢K♢7♢A♠Q)(♣J♠9♣7♡4)(♠3♢6♡Q♠4)(♠X♠K♢3♢9)(♠A♢4♢X♣2)(♡9♠8♣Q♢2)(♣5♢Q♡J♡X)(♡2♡7♡3♠5)(♠2♣K♢J♡6)(♠6♡5♣9♡A)
81
OK > (♢6♡Q♠4♢K)(♢4♣9♣X♡2)(♠2♡3♡5♡K)(♠Q♢5♡J♠9)(♣K♣J♡8♠X)(♣8♢2♣Q♡7)(♡4♢8♠A♣5)(♠K♡9♢A♢7)(♠7♢3♢J♣4)(♡A♡6♣2♠J)(♢9♠8♠6♣3)(♢X♠3♢Q♣A)(♣7♣6♠5♡X)
87
OK > (♠5♠X♢6♡4)(♣4♡2♡K♡Q)(♣5♣2♣3♣7)(♣K♣J♡5♣9)(♠2♡9♡A♠6)(♢A♢4♠J♠3)(♠4♠7♠9♢8)(♣A♢K♢7♡X)(♡6♠K♠8♡3)(♣X♢2♡8♢J)(♢9♢X♣Q♢3)(♠A♢Q♢5♣8)(♡J♠Q♣6♡7)
139
OK > (♢7♣Q♣J♠2)(♠7♠